In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model.logistic import LogisticRegression







## Regresion logística Vs Clasificador Bayesiano ingenuo

Compara los métodos de regresión logística y el clasificador bayesiano ingenuo en las siguientes tareas:



En primer lugar se leen los datos 

In [5]:
# Lectura de datos Spam

# Los datos de spam vienen con tipo de datos que no permiten hacer los
# calculos pertinentes para esto se hace arreglos 
#lectura de datos
spam = pd.read_csv("spam.csv")
data_spam = spam.to_numpy()

# Se hace para poner todos los datos como arrays, vienen compactos los datos
x = []
for i in range(len(data_spam)): 
    x.append(np.array([float(i) for i in data_spam[i][0].split()]))
x = np.array(x)

# Se definen los datos y los objetivos
x_data_s = x[:,:-1]
x_target_s = x[:,-1:]

print("Spam")
print("Tamaño de datos = {}, objetivos = {}".format(np.shape(x_data_s),np.shape(x_target_s)))

Tamaño de datos = (5171, 2000), objetivos = (5171, 1)


In [6]:
# Lectura datos cancer

# Definimos lista con los nombres de las características 
Names = ["ID ","Grosor del tumor","Uniformidad tamaño",
            "forma celula","Adhesion marginal","Tamaño celula epitelial",
            "Nucleos desnudos","Cromatina blanda","Núcleolos normales",
           "Mitosis de celulas","Clase"]
cancer = pd.read_csv("breast-cancer-wisconsin.data",names = Names)
# Eliminamos la columna de ID, no brinda información dado que no se repite
cancer = cancer.drop([cancer.columns[0]],axis = 1)
data_cancer = cancer.to_numpy()


# Para trabajar con uniformidad eliminamos los datos con "?" y asi poder comparar los 
# modelos, el objetivo es poder ver que algoritmo tiene mejor rendimiento 

index = np.where(data_cancer == "?")
data_cancer = np.delete(data_cancer,index,axis=0)
data_cancer[:,5] = data_cancer[:,5].astype(np.float)
data_cancer = data_cancer.astype(np.float)

x_data_c = data_cancer[:,:-1]
x_target_c = data_cancer[:,-1:]

print("Spam")
print("Tamaño de datos = {}, objetivos = {}".format(np.shape(x_data_c),np.shape(x_target_c)))

Spam
Tamaño de datos = (682, 9), objetivos = (682, 1)


Hacemos los modelos para cada uno de los problemas

* Clasificador de spam

In [34]:
# Clasificacion de spam

X_trainNBs, X_testNBs, Y_trainNBs,Y_testNBs = train_test_split(x_data_s,x_target_s,test_size = 0.3)
# Modelo Bayesiano ingenuo
spam_model_NB = MultinomialNB(alpha = 1e-10, class_prior = None, fit_prior=False)
spam_model_NB.fit(X_trainNBs,np.ravel(Y_trainNBs))
print("Rendimiento del modelo Naive Bayes = {:.3f}".format(spam_model_NB.score(X_testNBs,Y_testNBs)))
pred_prob_NB = spam_model_NB.predict(X_testNBs[1:2])

# Modelo de regresión logística
X_trainLRs, X_testLRs, Y_trainLRs,Y_testLRs = train_test_split(x_data_s,x_target_s,test_size = 0.3)
spam_model_LR = LogisticRegression(random_state=0,solver= "newton-cg")
spam_model_LR.fit(X_trainLRs,np.ravel(Y_trainLRs))
print("Rendimiento del modelo Regresión Logistica = {:.3f}".format(spam_model_LR.score(X_testLRs,Y_testLRs)))
pred_prob_LR = spam_model_LR.predict(X_testLRs[1:2])


Rendimiento del modelo Naive Bayes = 0.961
Rendimiento del modelo Regresión Logistica = 0.972


* Clasificador de tumores de Seno 

In [33]:
#Clasificacion de tumores de Seno

# Modelo Bayesiano ingenuo
X_trainNBc, X_testNBc, Y_trainNBc,Y_testNBc = train_test_split(x_data_c,x_target_c,test_size = 0.2)
cancer_model_G = GaussianNB()
cancer_model_G.fit(X_trainNBc,np.ravel(Y_trainNBc))
print("Rendimiento del modelo Naive Bayes = {:.3f}".format(cancer_model_G.score(X_testNBc,Y_testNBc)))
pred_prob_G = cancer_model_G.predict(X_testNBc[1:2])

#Modelo de regresion Logistica
X_trainLRc, X_testLRc, Y_trainLRc,Y_testLRc = train_test_split(x_data_c,x_target_c,test_size = 0.2)
cancer_model_G = LogisticRegression(random_state=0,solver= "liblinear")
cancer_model_G.fit(X_trainLRc,np.ravel(Y_trainLRc))
print("Rendimiento del modelo Naive Bayes = {:.3f}".format(cancer_model_G.score(X_testLRc,Y_testLRc)))
pred_prob_G = cancer_model_G.predict(X_testLRc[1:2])


Rendimiento del modelo Naive Bayes = 0.956
Rendimiento del modelo Naive Bayes = 0.964


Se hace la implementación de el modelo de scikit learn en el caso de regresión logistica para tener una refencia y comparar el modelo que se va a construir y ver si dista mucho del modelo ya programado correctamente

### Regresión logística para la clasificación de spam 

Para este modelo se tienen $N = 5171$ datos recopilados y $M = 2000$ características. A continuación debemos considerar dos clases, es o no spam

debemos encontrar la actualización de los parámetros del modelo a través de la siguiente expresión 

$$\vec{\omega}^{\tau+1} = \vec{\omega}^{\tau} - \mathcal{H}^{-1}\nabla_{\omega}E(\vec{\omega})$$

donde lo que debemos calcular es lo siguiente

$$\mathcal{H} = \Phi^{T}\mathcal{R}\Phi$$

$$\nabla_{\vec{\omega}}E(\vec{w}) = \Phi^{T}(\vec{y}-\vec{t})$$

In [64]:
#vector de objetivos
t = x_target_s
# Antes que nada se define la función sigmoide
def sigmoid(z):
    return 1/(1+np.exp(-z))

# Ahora se construye la matriz de diseño
unos = np.ones((len(x_data_s),1))
Phi = np.concatenate((unos,x_data_s),axis = 1)

# Definimos el vector de pesos
w = np.random.uniform(0,0.001,(1,np.shape(Phi)[1])).T

# calculamos el vector Y que estima los parametros desde la función sigmoide
Y = sigmoid(w.T@Phi.T).T

# calculamos el gradiente
G = Phi.T@(Y-t)

#Calculamos la matriz Hessiana
r = np.zeros((len(Y),len(Y)))
np.fill_diagonal(r,Y*(1-Y))
H = Phi.T@r@Phi


In [66]:
np.linalg.det(H[0:150,0:150])

6.9557094582990025e+277